In [1]:
# Import necessary libraries
from ultralytics import YOLO
from tensorflow.keras.models import load_model
import numpy as np
import tensorflow as tf
from PIL import Image as PILImage
import io
from IPython.display import Image, display
from PIL import Image as PILImage
import numpy as np
import tensorflow as tf
import io
import os
import pandas as pd
import torch
import json
from torch import float32, tensor
import torchmetrics
import cv2


2024-01-11 15:43:26.745321: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-11 15:43:26.745347: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-11 15:43:26.747456: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
import os
import shutil

# Source directory containing images
src_dir = './TRAIN'

# Destination directory to copy images
dst_dir = './TEST'

# Check if destination directory exists, create it if it doesn't
if not os.path.exists(dst_dir):
    os.makedirs(dst_dir)

# Loop through all files in source directory
for file_name in os.listdir(src_dir):
    # Check if file ends with .jpg
    if len(os.listdir(dst_dir)) == 50:
        break
    if file_name.endswith('.jpg'):
        # Construct full file paths
        src_path = os.path.join(src_dir, file_name)
        dst_path = os.path.join(dst_dir, file_name)
        # Copy file to destination directory
        shutil.copy(src_path, dst_path)




In [3]:
from transformers import TFViTForImageClassification

# Replace "your_fine_tuned_model_directory" with the actual directory where your fine-tuned model is stored.
model2 = TFViTForImageClassification.from_pretrained("./plantdoc_ViT_training/vit-base-patch16-224-in21k")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the YOLO model
yolo_model2 = YOLO('./YOLOv8_training/multiclass_best.pt').to(device)
yolo_model = YOLO('./YOLOv8_training/best.pt').to(device)
# Class mapping
# class_mapping = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}

# Directory containing images to predict
directory = './datasets(1)/test/images'
# directory = './TEST'

All model checkpoint layers were used when initializing TFViTForImageClassification.

All the layers of TFViTForImageClassification were initialized from the model checkpoint at ./plantdoc_ViT_training/vit-base-patch16-224-in21k.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTForImageClassification for predictions without further training.


In [4]:
class_mapping = {}

# Assuming your dataset is a pandas DataFrame named 'df' and the column with class labels is 'class'
for i in range(29):
    predicted_class_label = model2.config.id2label[i]
    class_mapping[predicted_class_label] = i

print("Class Mapping:", class_mapping)

Class Mapping: {'Soyabean leaf': 0, 'Tomato leaf late blight': 1, 'Corn Gray leaf spot': 2, 'Cherry leaf': 3, 'Tomato Septoria leaf spot': 4, 'Tomato two spotted spider mites leaf': 5, 'Potato leaf late blight': 6, 'Tomato leaf mosaic virus': 7, 'Corn rust leaf': 8, 'Squash Powdery mildew leaf': 9, 'Bell_pepper leaf': 10, 'Apple leaf': 11, 'Potato leaf early blight': 12, 'grape leaf black rot': 13, 'Strawberry leaf': 14, 'Apple rust leaf': 15, 'Tomato mold leaf': 16, 'Tomato leaf bacterial spot': 17, 'Corn leaf blight': 18, 'Blueberry leaf': 19, 'grape leaf': 20, 'Tomato leaf yellow virus': 21, 'Raspberry leaf': 22, 'Bell_pepper leaf spot': 23, 'Tomato Early blight leaf': 24, 'Potato leaf': 25, 'Peach leaf': 26, 'Tomato leaf': 27, 'Apple Scab Leaf': 28}


In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
preds=[]
# Define a list of dictionaries
data = []
# Create empty lists to store results
resized_cropped_images = []
predicted_classes = []

In [7]:
from transformers import ViTFeatureExtractor
import torch
import pandas as pd
import tensorflow as tf
from PIL import Image as PILImage
import os
import io
from torchvision.ops.boxes import box_iou

# Load the ViT feature extractor
feature_extractor = ViTFeatureExtractor.from_pretrained('./plantdoc_ViT_training/vit-base-patch16-224-in21k')

# Load the YOLO results dataframe
df = pd.read_csv('./YOLOv8_training/new_labels.csv')

# Define the target size for ResNet50
target_size = (224, 224)

# Initialize arrays for predictions and targets
all_preds = []
all_targets = []

# IoU threshold for matching predictions with ground truth
iou_threshold = 0.5

# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results = yolo_model.predict(os.path.join(directory, filename))
    for x in results:
        temp_preds = []
        temp_targets = []
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

#             # Resize the cropped image to the target size
#             resized_cropped_im = cropped_im.resize(target_size)

#             # Save the resized and cropped image to a BytesIO object
#             img_io = io.BytesIO()
#             resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list (if needed)
            # resized_cropped_images.append(resized_cropped_im)

            # Extract features using ViT
            inputs = feature_extractor(images=cropped_im)
            pixel_values_tensor = tf.convert_to_tensor(inputs['pixel_values'], dtype=tf.float32)
            outputs = model2(pixel_values=pixel_values_tensor)
            logits = outputs.logits
            logits = tf.nn.softmax(logits)
            
            # model predicts one of the 1000 ImageNet classes
            predicted_class_index = tf.argmax(logits, axis=-1).numpy().item()
            predicted_class_label = model2.config.id2label[predicted_class_index]
            
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([predicted_class_index]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = torch.tensor([tf.reduce_max(logits).numpy()]).to(device) * r.boxes.conf

            # Append bounding box details to temp_preds
            temp_preds.append({'id': filename, 'boxes': bbox_tensor, 'labels': label_tensor, 'scores': score_tensor})
            for index, row in filtered_df.iterrows():
                temp_targets.append({'id': filename,'boxes': torch.tensor([[row['xmin'], row['ymin'], row['xmax'], row['ymax']]]).to(device),
                                    'labels': torch.tensor([class_mapping[row['class']]]).to(device)})

            # Calculate IoU matrix between prediction and ground truth boxes
            iou_matrix = box_iou(torch.stack([r['boxes'].squeeze(0) for r in temp_preds]),
                                 torch.stack([t['boxes'].squeeze(0) for t in temp_targets]))

            # Find the index of the ground truth box with the highest IoU for each prediction
            max_iou_indices = torch.argmax(iou_matrix, dim=1)

            # Update temp_targets based on the highest IoU
            temp_targets = [temp_targets[i] for i in max_iou_indices]
            all_preds.extend(temp_preds)
            all_targets.extend(temp_targets)

/home/suryansh/anaconda3/lib/python3.11/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1309.jpg: 640x416 1 leaf, 55.1ms
Speed: 1.1ms preprocess, 55.1ms inference, 194.1ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2458.jpg: 480x640 12 leafs, 61.8ms
Speed: 1.0ms preprocess, 61.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1086.jpg: 416x640 2 leafs, 53.2ms
Speed: 1.1ms preprocess, 53.2ms inference, 0.5ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img952.jpg: 448x640 3 leafs, 56.6ms
Speed: 1.3ms preprocess, 56.6ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img909.jp

Speed: 1.5ms preprocess, 13.1ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img756.jpg: 448x640 2 leafs, 12.9ms
Speed: 1.6ms preprocess, 12.9ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1110.jpg: 416x640 1 leaf, 12.4ms
Speed: 1.7ms preprocess, 12.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img354.jpg: 480x640 1 leaf, 10.1ms
Speed: 1.5ms preprocess, 10.1ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img889.jpg: 480x640 7 leafs, 9.6ms
Speed: 1.8ms preprocess, 9.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1

Speed: 1.7ms preprocess, 14.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1454.jpg: 480x640 3 leafs, 9.8ms
Speed: 1.5ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img82.jpg: 640x480 1 leaf, 14.8ms
Speed: 1.2ms preprocess, 14.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2426.jpg: 544x640 4 leafs, 14.8ms
Speed: 1.9ms preprocess, 14.8ms inference, 1.4ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img569.jpg: 480x640 1 leaf, 14.7ms
Speed: 1.4ms preprocess, 14.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1

Speed: 1.0ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img456.jpg: 640x640 7 leafs, 16.3ms
Speed: 1.7ms preprocess, 16.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img497.jpg: 480x640 8 leafs, 9.8ms
Speed: 1.6ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1156.jpg: 256x640 1 leaf, 55.6ms
Speed: 1.3ms preprocess, 55.6ms inference, 1.1ms postprocess per image at shape (1, 3, 256, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1997.jpg: 512x640 1 leaf, 13.6ms
Speed: 1.7ms preprocess, 13.6ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)

image 1/1

Speed: 1.4ms preprocess, 9.1ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img19.jpg: 640x384 1 leaf, 11.5ms
Speed: 1.4ms preprocess, 11.5ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img879.jpg: 640x448 2 leafs, 12.3ms
Speed: 1.2ms preprocess, 12.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 448)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2078.jpg: 416x640 4 leafs, 12.3ms
Speed: 1.1ms preprocess, 12.3ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2121.jpg: 512x640 7 leafs, 12.9ms
Speed: 1.7ms preprocess, 12.9ms inference, 1.3ms postprocess per image at shape (1, 3, 512, 640)

image 1

Speed: 1.4ms preprocess, 14.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2194.jpg: 448x640 2 leafs, 12.8ms
Speed: 1.3ms preprocess, 12.8ms inference, 0.8ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1457.jpg: 640x512 5 leafs, 64.1ms
Speed: 1.9ms preprocess, 64.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 512)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1752.jpg: 640x480 8 leafs, 9.5ms
Speed: 1.3ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img237.jpg: 480x640 6 leafs, 14.1ms
Speed: 1.5ms preprocess, 14.1ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

image

Speed: 1.5ms preprocess, 13.0ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img501.jpg: 480x640 2 leafs, 9.7ms
Speed: 0.9ms preprocess, 9.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2334.jpg: 480x640 3 leafs, 9.5ms
Speed: 1.1ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1535.jpg: 640x544 1 leaf, 9.2ms
Speed: 1.2ms preprocess, 9.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 544)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img392.jpg: 448x640 10 leafs, 8.0ms
Speed: 1.3ms preprocess, 8.0ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /h

In [8]:
import torch
import pandas as pd
import tensorflow as tf
from PIL import Image as PILImage
import os
import io
from torchvision.ops.boxes import box_iou
# Class mapping
class_mapping2 = {'Apple Scab Leaf': 0, 'Apple leaf': 1, 'Apple rust leaf': 2, 'Bell_pepper leaf': 3, 'Bell_pepper leaf spot': 4, 'Blueberry leaf': 5, 'Cherry leaf': 6, 'Corn Gray leaf spot': 7, 'Corn leaf blight': 8, 'Corn rust leaf': 9, 'Peach leaf': 10, 'Potato leaf': 11, 'Potato leaf early blight': 12, 'Potato leaf late blight': 13, 'Raspberry leaf': 14, 'Soyabean leaf': 15, 'Squash Powdery mildew leaf': 16, 'Strawberry leaf': 17, 'Tomato Early blight leaf': 18, 'Tomato Septoria leaf spot': 19, 'Tomato leaf': 20, 'Tomato leaf bacterial spot': 21, 'Tomato leaf late blight': 22, 'Tomato leaf mosaic virus': 23, 'Tomato leaf yellow virus': 24, 'Tomato mold leaf': 25, 'Tomato two spotted spider mites leaf': 26, 'grape leaf': 27, 'grape leaf black rot': 28}
# Load the YOLO results dataframe
df = pd.read_csv('./YOLOv8_training/new_labels.csv')

# Define the target size for ResNet50
target_size = (224, 224)

# Initialize arrays for predictions and targets
all_preds2 = []
all_targets2 = []

# IoU threshold for matching predictions with ground truth
iou_threshold = 0.8

# Process YOLO results
for filename in os.listdir(directory):
    filtered_df = df[df['filename'] == filename]
    results2 = yolo_model2.predict(os.path.join(directory, filename))
    for x in results2:
        temp_preds2 = []
        temp_targets2 = []
        for r in x:
            # Get the image and bounding box
            im_array = r.plot()  # plot a BGR numpy array of predictions
            im = PILImage.fromarray(im_array[..., ::-1])  # Convert BGR to RGB PIL image
            bbox = r.boxes.data[0, :4].cpu().numpy()  # Get the bounding box

            # Crop the detected object from the image
            cropped_im = im.crop((bbox[0], bbox[1], bbox[2], bbox[3]))

            # Resize the cropped image to the target size
            resized_cropped_im = cropped_im.resize(target_size)
            # plt.imshow(resized_cropped_im)

            # Save the resized and cropped image to a BytesIO object
            img_io = io.BytesIO()
            resized_cropped_im.save(img_io, 'PNG')

            # Append the resized and cropped image to the list
            resized_cropped_images.append(resized_cropped_im)
            predicted_class_index = r.boxes.cls
            predicted_class_label = list(class_mapping.keys())[int(predicted_class_index.cpu().item())]
            predicted_classes.append(predicted_class_label)
            # Convert bounding box to tensor and move to GPU
            bbox_tensor = torch.tensor([[bbox[0], bbox[1], bbox[2], bbox[3]]]).to(device)
            
            # Convert labels to tensor and move to GPU
            label_tensor = torch.tensor([int(predicted_class_index)]).to(device)

            # Convert scores to tensor and move to GPU
            score_tensor = r.boxes.conf

            # Append bounding box details to temp_preds
            temp_preds2.append({'id':filename, 'boxes': bbox_tensor, 'labels': label_tensor, 'scores': score_tensor})
            for index, row in filtered_df.iterrows():
                temp_targets2.append({'id': filename,'boxes': torch.tensor([[row['xmin'], row['ymin'], row['xmax'], row['ymax']]]).to(device),
                                    'labels': torch.tensor([class_mapping2[row['class']]]).to(device)})

            # Calculate IoU matrix between prediction and ground truth boxes
            iou_matrix = box_iou(torch.stack([r['boxes'].squeeze(0) for r in temp_preds2]),
                                 torch.stack([t['boxes'].squeeze(0) for t in temp_targets2]))

            # Find the index of the ground truth box with the highest IoU for each prediction
            max_iou_indices = torch.argmax(iou_matrix, dim=1)

            # Update temp_targets based on the highest IoU
            temp_targets2 = [temp_targets2[i] for i in max_iou_indices]
            all_preds2.extend(temp_preds2)
            all_targets2.extend(temp_targets2)


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1309.jpg: 640x416 1 Corn leaf blight, 12.7ms
Speed: 1.4ms preprocess, 12.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 416)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2458.jpg: 480x640 7 Tomato leaf mosaic viruss, 15.2ms
Speed: 1.2ms preprocess, 15.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1086.jpg: 416x640 1 grape leaf black rot, 13.2ms
Speed: 1.6ms preprocess, 13.2ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img952.jpg: 448x640 3 Cherry leafs, 13.0ms
Speed: 1.6ms preprocess, 13.0ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img316.jpg: 288x640 6 Tomato Septoria leaf spots, 10.0ms
Speed: 0.8ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img309.jpg: 448x640 2 Apple rust leafs, 4 Bell_pepper leaf spots, 13.1ms
Speed: 1.2ms preprocess, 13.1ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img551.jpg: 512x640 1 Apple rust leaf, 13.4ms
Speed: 1.5ms preprocess, 13.4ms inference, 1.2ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img756.jpg: 448x640 1 Squash Powdery mildew leaf, 13.5ms
Speed: 1.0ms preprocess, 13.5ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /hom

Speed: 1.0ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1548.jpg: 544x640 1 Corn Gray leaf spot, 1 Corn leaf blight, 9.3ms
Speed: 1.0ms preprocess, 9.3ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1352.jpg: 544x640 5 Raspberry leafs, 8.9ms
Speed: 1.0ms preprocess, 8.9ms inference, 0.5ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2345.jpg: 640x480 2 Raspberry leafs, 9.5ms
Speed: 0.9ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1453.jpg: 480x640 1 Corn Gray leaf spot, 9.9ms
Speed: 0.9ms preprocess, 9.9ms inference

Speed: 1.0ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img2386.jpg: 640x640 3 Potato leaf early blights, 1 Potato leaf late blight, 1 Soyabean leaf, 1 Tomato leaf late blight, 11.7ms
Speed: 1.0ms preprocess, 11.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img397.jpg: 480x640 1 Blueberry leaf, 2 Cherry leafs, 2 Strawberry leafs, 14.4ms
Speed: 1.3ms preprocess, 14.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img530.jpg: 544x640 1 Potato leaf late blight, 1 Tomato leaf late blight, 14.2ms
Speed: 1.3ms preprocess, 14.2ms inference, 1.0ms postprocess per image at shape (1, 3, 544, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-d

Speed: 0.9ms preprocess, 9.5ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1441.jpg: 480x640 1 Tomato leaf late blight, 9.9ms
Speed: 0.9ms preprocess, 9.9ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img222.jpg: 640x608 3 Strawberry leafs, 9.4ms
Speed: 1.5ms preprocess, 9.4ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 608)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1893.jpg: 448x640 4 Soyabean leafs, 2 grape leaf black rots, 8.3ms
Speed: 1.8ms preprocess, 8.3ms inference, 0.5ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1533.jpg: 480x640 1 Corn rust leaf, 9.7ms
Speed: 1.1ms preprocess, 9.7ms infere

Speed: 0.9ms preprocess, 8.1ms inference, 0.6ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img562.jpg: 512x640 6 Strawberry leafs, 9.0ms
Speed: 0.9ms preprocess, 9.0ms inference, 0.5ms postprocess per image at shape (1, 3, 512, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1021.jpg: 384x640 1 Tomato leaf late blight, 7.6ms
Speed: 0.8ms preprocess, 7.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1792.jpg: 640x640 3 Tomato Septoria leaf spots, 11.8ms
Speed: 1.2ms preprocess, 11.8ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img42.jpg: 480x640 4 Tomato Septoria leaf spots, 9.9ms
Speed: 1.0ms preprocess, 9.9ms inferen

Speed: 0.6ms preprocess, 9.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1024.jpg: 480x640 2 Tomato Early blight leafs, 9.8ms
Speed: 0.8ms preprocess, 9.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1131.jpg: 480x640 3 Tomato mold leafs, 9.6ms
Speed: 1.1ms preprocess, 9.6ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1613.jpg: 480x640 1 Tomato leaf, 2 Tomato leaf late blights, 9.6ms
Speed: 1.6ms preprocess, 9.6ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1163.jpg: 480x640 12 Tomato leaf late blights, 9.8ms
Speed: 1.1ms preproces


image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img1697.jpg: 640x480 2 Cherry leafs, 14.0ms
Speed: 1.4ms preprocess, 14.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img837.jpg: 576x640 1 Squash Powdery mildew leaf, 14.1ms
Speed: 1.3ms preprocess, 14.1ms inference, 1.2ms postprocess per image at shape (1, 3, 576, 640)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img599.jpg: 640x320 2 Tomato Septoria leaf spots, 11.8ms
Speed: 0.8ms preprocess, 11.8ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 /home/suryansh/Documents/GitHub/Plant-disease-detection/datasets(1)/test/images/img9.jpg: 448x640 3 Tomato leaf mosaic viruss, 12.8ms
Speed: 1.2ms preprocess, 12.8ms inference, 1.2ms postprocess per image at shape (1, 3, 448, 640)

image 1/1 /home/suryansh/Documents/

In [9]:
import torch
from torchvision.ops.boxes import nms

# Assuming you have a list of predictions from two YOLO models stored in all_preds and all_preds2
# Each element in the list is a dictionary with keys 'id', 'boxes', 'labels', and 'scores'
# Also assuming you have target labels stored in all_targets and all_targets2

# Concatenate the results from both models
# Concatenate the results from both models
all_results = {
    'id': [pred['id'] for pred in all_preds] + [pred['id'] for pred in all_preds2],
    'boxes': torch.cat([pred['boxes'].clone().detach() for pred in all_preds] + [pred['boxes'].clone().detach() for pred in all_preds2], dim=0),
    'labels': torch.cat([pred['labels'].clone().detach() for pred in all_preds] + [pred['labels'].clone().detach() for pred in all_preds2], dim=0),
    'scores': torch.cat([pred['scores'].clone().detach() for pred in all_preds] + [pred['scores'].clone().detach() for pred in all_preds2], dim=0)
}

# Concatenate the target lists from both models
all_final_targets = {
    'id': [target['id'] for target in all_targets] + [target['id'] for target in all_targets2],
    'boxes': torch.cat([target['boxes'].clone().detach() for target in all_targets] + [target['boxes'].clone().detach() for target in all_targets2], dim=0),
    'labels': torch.cat([target['labels'].clone().detach() for target in all_targets] + [target['labels'].clone().detach() for target in all_targets2], dim=0)
}


# Set the IoU threshold for NMS
iou_threshold = 0.8

# Perform NMS for each unique ID
unique_ids = list(set(all_results['id']))
final_filtered_results = []
final_filtered_targets = []

for unique_id in unique_ids:
    # Filter predictions and targets for the current unique ID
    mask_results = [i for i, id in enumerate(all_results['id']) if id == unique_id]
    mask_targets = [i for i, id in enumerate(all_final_targets['id']) if id == unique_id]

    # Apply NMS on the subset of predictions for the current unique ID
    keep_indices = nms(all_results['boxes'][mask_results], all_results['scores'][mask_results], iou_threshold)

    # Filter predictions and targets based on NMS results
    final_filtered_results.extend([
        {
            'id': unique_id,
            'boxes': all_results['boxes'][mask_results][keep_indices],
            'labels': all_results['labels'][mask_results][keep_indices],
            'scores': all_results['scores'][mask_results][keep_indices],
        }
    ])

    final_filtered_targets.extend([
        {
            'id': unique_id,
            'boxes': all_final_targets['boxes'][mask_targets][keep_indices],
            'labels': all_final_targets['labels'][mask_targets][keep_indices],
        }
    ])
# Display the final filtered results
print("Final Filtered Predictions:")
formatted_results = []
formatted_targets = []
for result in final_filtered_results:
    formatted_result = [{
        'boxes': tensor(result['boxes'].cpu().numpy().tolist()).to(device),
        'labels': tensor(result['labels'].cpu().numpy().tolist()).to(device),
        'scores': tensor(result['scores'].cpu().numpy().tolist()).to(device)
    }]
    formatted_results.extend(formatted_result)
    
for result in final_filtered_targets:
    formatted_targets = [{
        'boxes': tensor(result['boxes'].cpu().numpy().tolist()).to(device),
        'labels': tensor(result['labels'].cpu().numpy().tolist()).to(device),
    }]
    formatted_targets.extend(formatted_result)
# Store the formatted results in final_filtered_results
final_filtered_results = formatted_results
print(final_filtered_results)

Final Filtered Predictions:
[{'boxes': tensor([[ 430.3079,   46.1207, 1115.0576, 1036.1008]], device='cuda:0'), 'labels': tensor([4], device='cuda:0'), 'scores': tensor([0.8459], device='cuda:0')}, {'boxes': tensor([[0.0000e+00, 1.2109e+00, 5.7212e+02, 3.4715e+02],
        [6.1668e+02, 2.2338e-02, 7.5000e+02, 1.3444e+02]], device='cuda:0'), 'labels': tensor([16,  0], device='cuda:0'), 'scores': tensor([0.9668, 0.2287], device='cuda:0')}, {'boxes': tensor([[  0.0000, 160.5174, 208.9548, 284.2183],
        [428.7725,  90.2564, 565.3612, 248.9724],
        [238.8710,  19.7182, 340.7936, 225.7074],
        [305.9838, 293.8942, 437.3342, 479.8495],
        [130.5516,   0.0000, 266.3249, 218.1907],
        [390.6857, 237.8558, 476.9861, 406.9052],
        [  0.0000,  29.9494, 121.2516, 258.9439],
        [207.9481, 228.1163, 349.2122, 431.5228],
        [348.0801,  32.7825, 437.6999, 180.3595],
        [366.2124, 158.4264, 427.7374, 230.4000],
        [  0.0000, 352.8680,  56.3497, 480.0000]

In [10]:
# for img_data, predicted_class in zip(resized_cropped_images, predicted_classes):
#     display(img_data, predicted_class, class_mapping[predicted_class])


In [11]:
metrics = torchmetrics.detection.MeanAveragePrecision().to(device)
for i in range(len(final_filtered_results)):
    print([final_filtered_results[i]], [final_filtered_targets[i]])
metrics.update(final_filtered_results, final_filtered_targets)
print(metrics.compute())

[{'boxes': tensor([[ 430.3079,   46.1207, 1115.0576, 1036.1008]], device='cuda:0'), 'labels': tensor([4], device='cuda:0'), 'scores': tensor([0.8459], device='cuda:0')}] [{'id': 'img660.jpg', 'boxes': tensor([[ 410,   78, 1104, 1040]], device='cuda:0'), 'labels': tensor([4], device='cuda:0')}]
[{'boxes': tensor([[0.0000e+00, 1.2109e+00, 5.7212e+02, 3.4715e+02],
        [6.1668e+02, 2.2338e-02, 7.5000e+02, 1.3444e+02]], device='cuda:0'), 'labels': tensor([16,  0], device='cuda:0'), 'scores': tensor([0.9668, 0.2287], device='cuda:0')}] [{'id': 'img1271.jpg', 'boxes': tensor([[ 22,   1, 568, 346],
        [ 22,   1, 568, 346]], device='cuda:0'), 'labels': tensor([16,  9], device='cuda:0')}]
[{'boxes': tensor([[  0.0000, 160.5174, 208.9548, 284.2183],
        [428.7725,  90.2564, 565.3612, 248.9724],
        [238.8710,  19.7182, 340.7936, 225.7074],
        [305.9838, 293.8942, 437.3342, 479.8495],
        [130.5516,   0.0000, 266.3249, 218.1907],
        [390.6857, 237.8558, 476.9861, 406

[{'boxes': tensor([[ 34.9698, 130.4425, 589.1766, 401.3817],
        [336.6619, 255.8500, 646.4916, 600.0000],
        [337.1588,   0.0000, 621.1578, 124.7071]], device='cuda:0'), 'labels': tensor([ 0, 11, 28], device='cuda:0'), 'scores': tensor([0.5439, 0.4477, 0.3678], device='cuda:0')}] [{'id': 'img1226.jpg', 'boxes': tensor([[ 34, 146, 559, 387],
        [ 34, 146, 559, 387],
        [ 34, 146, 559, 387]], device='cuda:0'), 'labels': tensor([ 2, 15, 15], device='cuda:0')}]
[{'boxes': tensor([[7.1379e+01, 2.7626e+01, 2.4121e+02, 3.1737e+02],
        [2.6315e+02, 5.7867e+01, 4.2438e+02, 3.0807e+02],
        [4.0863e+02, 3.1302e-01, 4.9988e+02, 1.7062e+02]], device='cuda:0'), 'labels': tensor([22, 13, 27], device='cuda:0'), 'scores': tensor([0.7888, 0.7834, 0.5790], device='cuda:0')}] [{'id': 'img2452.jpg', 'boxes': tensor([[ 71,  19, 246, 321],
        [269,  67, 426, 307],
        [269,  67, 426, 307]], device='cuda:0'), 'labels': tensor([13, 13,  6], device='cuda:0')}]
[{'boxes': t

{'map': tensor(0.3443), 'map_50': tensor(0.4677), 'map_75': tensor(0.3908), 'map_small': tensor(0.), 'map_medium': tensor(0.3916), 'map_large': tensor(0.3530), 'mar_1': tensor(0.2077), 'mar_10': tensor(0.4197), 'mar_100': tensor(0.4260), 'mar_small': tensor(0.), 'mar_medium': tensor(0.4235), 'mar_large': tensor(0.4315), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], dtype=torch.int32)}


In [12]:
# Assuming you have final_filtered_results and all_preds as lists of dictionaries
# Each dictionary has keys 'boxes', 'labels', and 'scores'

# Criteria for matching (you can modify this based on your requirements)
def is_match(pred1, pred2):
    return (
        torch.equal(pred1['boxes'], pred2['boxes']) and
        torch.equal(pred1['labels'], pred2['labels']) and
        torch.equal(pred1['scores'], pred2['scores'])
    )

# Find common items
common_items = [
    final_pred for final_pred in final_filtered_results
    for all_pred in all_preds
    if is_match(final_pred, all_pred)
]

# Display common items
print("Common Items:")
print(common_items)


Common Items:
[{'boxes': tensor([[  0.0000,   0.0000, 640.0000, 370.8004]], device='cuda:0'), 'labels': tensor([18], device='cuda:0'), 'scores': tensor([0.6392], device='cuda:0')}, {'boxes': tensor([[  0.0000,  69.7315, 822.5880, 848.6031]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.5266], device='cuda:0')}, {'boxes': tensor([[  0.6311,   6.3921, 555.9023, 324.4828]], device='cuda:0'), 'labels': tensor([3], device='cuda:0'), 'scores': tensor([0.6984], device='cuda:0')}, {'boxes': tensor([[  7.3174,  11.0949, 240.0000, 179.5429]], device='cuda:0'), 'labels': tensor([23], device='cuda:0'), 'scores': tensor([0.7319], device='cuda:0')}, {'boxes': tensor([[1073.8007,  266.2285, 1447.4565,  830.5066]], device='cuda:0'), 'labels': tensor([11], device='cuda:0'), 'scores': tensor([0.7247], device='cuda:0')}, {'boxes': tensor([[   0.0000,    0.0000, 2834.9307, 4260.6050]], device='cuda:0'), 'labels': tensor([8], device='cuda:0'), 'scores': tensor([0.6607], dev

In [13]:
print(len(all_preds))
print(len(all_preds2))
print(len(common_items))

5486
4752
17
